## REACT

In [1]:
import pandas as pd
import os
from openai import OpenAI
from new_keyword_pipeline import setup_vectordb, search_top10_keywords, generate_new_keyword
from react_converted import main as sim_prompt_main

os.environ["OPENAI_API_KEY"] = "sk-"
client = OpenAI()
keyword_vectordb = setup_vectordb("./keyword_list.csv")

input = {
    "components": "로봇청소기",
    "afterchange": "ThinkQ 평면도상에 선을 그어 청소구역을 지정하도록 해주세요"
}

# sim_prompt_main에서 ICC/Proposal/유사도/키워드 추출
ticket_id, components, before_change, after_change, generated_summary, ragas_score, keyword, timing_results, proposal_summary_all = sim_prompt_main(input)

print(f"\n\n✅ [최종 결과]\n"
      f"ticket_id: {ticket_id}\n"
      f"components: {components}\n"
      f"beforechange: {before_change}\n"
      f"afterchange: {after_change}\n"
      f"gen_sum: {generated_summary}\n"
      f"RAGAS_score: {ragas_score}\n"
      f"sim_keyword: {keyword}")

# Proposal 분기 및 처리
if ragas_score >= 90:
    print(f"[{ticket_id}] 유사도 90↑ 기존 키워드 업로드: {keyword}")
else:
    top10_keywords = search_top10_keywords(keyword_vectordb, generated_summary, top_k=10)
    result = generate_new_keyword(
        client, ticket_id, components, before_change, after_change,
        generated_summary, ragas_score, keyword, top10_keywords
    )
    print(f"[{ticket_id}] 유사도 90↓ 새 키워드 생성: {result['new_keyword']}")

/Users/yunho/opt/anaconda3/envs/llm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/yunho/opt/anaconda3/envs/llm/lib/python3.11/site-packages/sklearn/utils/_param_validation.py:14: UserWarning: A NumPy version >=1.22.4 and <2.3.0 is required for this version of SciPy (detected version 2.3.1)
  from scipy.sparse import csr_matrix, issparse
/Users/yunho/Documents/up_phase2/react_converted.py:84: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  original_vectordb = FAISS.from_documents(documents

✅ 벡터 DB 구축 완료! 저장된 문서 수: 936

📥 입력 제안문:
ThinkQ 평면도상에 선을 그어 청소구역을 지정하도록 해주세요
⏱️ [입력받기] 시작
✅ [입력받기] 완료 - 소요 시간: 0.00초
⏱️ [ICC 분류] 시작
✅ [ICC 분류] 완료 - 소요 시간: 0.39초
⏱️ [ICC 분기처리] 시작
👉 판별 결과 : Proposal
✅ [ICC 분기처리] 완료 - 소요 시간: 0.00초
⏱️ [제안 분리] 시작
✅ [제안 분리] 완료 - 소요 시간: 0.65초
⏱️ [전체 제안 요약] 시작
✅ [전체 제안 요약] 완료 - 소요 시간: 0.68초
⏱️ [첫번째 제안 선택] 시작
✅ [첫번째 제안 선택] 완료 - 소요 시간: 0.66초
⏱️ [Top-10 유사 문서 검색] 시작
✅ [Top-10 유사 문서 검색] 완료 - 소요 시간: 0.84초
⏱️ [Self-Consistency 평가] 시작
✅ [Self-Consistency 평가] 완료 - 소요 시간: 5.81초
⏱️ [Self-Consistency 파싱] 시작
✅ [Self-Consistency 파싱] 완료 - 소요 시간: 0.00초
⏱️ [Self-Consistency 테이블 포맷] 시작
✅ [Self-Consistency 테이블 포맷] 완료 - 소요 시간: 0.00초
⏱️ [최종 결정] 시작
✅ [최종 결정] 완료 - 소요 시간: 10.92초
✅ result keys: dict_keys(['components', 'afterchange', 'icc_check', 'proposals', 'proposal_summary_all', 'first_proposal', 'top_10_table', 'top_10_docs', 'top_10_records', 'self_consistency_responses', 'parsed_self_consistency', 'self_consistency_table', 'final_result'])

📄 전체 제안 요약:
['Enable the ThinkQ robot

In [ ]:
# {'components': '로봇청소기', 
#  'generated_summary': 'ThinkQ 평면도상에 선을 그어 청소구역을 지정하도록 해주세요', 
#  'icc_check': 'Suggestion', 
#  'proposals': '["ThinkQ 평면도상에 선을 그어 청소구역을 지정하도록 해주세요"]', 
#  'proposal_summary_all': ['Enable the ThinkQ robot vacuum to draw lines on a floor plan to designate cleaning zones.'], 
#  'first_proposal': 'Enable the ThinkQ robot vacuum to draw lines on a floor plan to designate cleaning zones.', 
#  'top_10_table': '| 순위 | Ticket ID | Cosine 유사도 | 문서 요약 | 주요 키워드 |\n| --- | --- | --- | --- | --- |\n| 1 | 569aa41 | 0.7813 | The user suggests having the control panel light up when the dryer is touched anywhere to improve vi | |\n| 2 | 4d1e216 | 0.7796 | The user suggests adding an option to indicate when the lint screen has been cleaned to make the rem | |\n| 3 | e35a181 | 0.7730 | The user suggests adding a button that allows them to confirm they have cleaned the filter, so the r | |\n| 4 | a299512 | 0.7669 | The user suggests being able to control the light inside and turn it on and off as needed. | |\n| 5 | 16276a5 | 0.7629 | The user suggests changing the default settings for normal and heavy duty cycles to allow adjustment | |\n| 6 | 1d60287 | 0.7613 | The user suggests being able to select low heat and use sensor drying. | |\n| 7 | 6635aa0 | 0.7608 | The user suggests that the app should include descriptions of the available cycles and possibly the  | |\n| 8 | b6452d1 | 0.7595 | The user suggests enabling remote start without needing to push the start button on the appliance fi | |\n| 9 | f11cf93 | 0.7581 | The user suggests allowing the normal mode to be customized to include wrinkle prevention all the ti | |\n| 10 | 0b178a7 | 0.7580 | The user suggests adding a feature to the drying cycle that allows choosing both temperature and dry | |', 
#  'top_10_docs': [Document(metadata={'ticket_id': '569aa41', 'doc_id': 'doc_0'}, page_content='The user suggests having the control panel light up when the dryer is touched anywhere to improve visibility in a dark laundry room.'), Document(metadata={'ticket_id': '4d1e216', 'doc_id': 'doc_14'}, page_content='The user suggests adding an option to indicate when the lint screen has been cleaned to make the reminder more accurate and useful.'), Document(metadata={'ticket_id': 'e35a181', 'doc_id': 'doc_12'}, page_content='The user suggests adding a button that allows them to confirm they have cleaned the filter, so the reminder message to clean the filter after 10 cycles will stop until the next cycle reset.'), Document(metadata={'ticket_id': 'a299512', 'doc_id': 'doc_15'}, page_content='The user suggests being able to control the light inside and turn it on and off as needed.'), Document(metadata={'ticket_id': '16276a5', 'doc_id': 'doc_37'}, page_content="The user suggests changing the default settings for normal and heavy duty cycles to allow adjustment of the dry level, specifically setting the dry level to 'very' for the heavy duty cycle to ensure cotton throw rugs are adequately dried."), Document(metadata={'ticket_id': '1d60287', 'doc_id': 'doc_34'}, page_content='The user suggests being able to select low heat and use sensor drying.'), Document(metadata={'ticket_id': '6635aa0', 'doc_id': 'doc_25'}, page_content='The user suggests that the app should include descriptions of the available cycles and possibly the settings that are programmed for each cycle.'), Document(metadata={'ticket_id': 'b6452d1', 'doc_id': 'doc_3'}, page_content='The user suggests enabling remote start without needing to push the start button on the appliance first.'), Document(metadata={'ticket_id': 'f11cf93', 'doc_id': 'doc_16'}, page_content='The user suggests allowing the normal mode to be customized to include wrinkle prevention all the time instead of requiring prior knowledge of its need.'), Document(metadata={'ticket_id': '0b178a7', 'doc_id': 'doc_30'}, page_content='The user suggests adding a feature to the drying cycle that allows choosing both temperature and dry level.')], 
#  'top_10_records': [{'순위': 1, 'ticket_id': '569aa41', 'Cosine 유사도': np.float64(0.7813), '문서 요약': 'The user suggests having the control panel light up when the dryer is touched anywhere to improve vi'}, {'순위': 2, 'ticket_id': '4d1e216', 'Cosine 유사도': np.float64(0.7797), '문서 요약': 'The user suggests adding an option to indicate when the lint screen has been cleaned to make the rem'}, {'순위': 3, 'ticket_id': 'e35a181', 'Cosine 유사도': np.float64(0.773), '문서 요약': 'The user suggests adding a button that allows them to confirm they have cleaned the filter, so the r'}, {'순위': 4, 'ticket_id': 'a299512', 'Cosine 유사도': np.float64(0.767), '문서 요약': 'The user suggests being able to control the light inside and turn it on and off as needed.'}, {'순위': 5, 'ticket_id': '16276a5', 'Cosine 유사도': np.float64(0.7629), '문서 요약': 'The user suggests changing the default settings for normal and heavy duty cycles to allow adjustment'}, {'순위': 6, 'ticket_id': '1d60287', 'Cosine 유사도': np.float64(0.7613), '문서 요약': 'The user suggests being able to select low heat and use sensor drying.'}, {'순위': 7, 'ticket_id': '6635aa0', 'Cosine 유사도': np.float64(0.7608), '문서 요약': 'The user suggests that the app should include descriptions of the available cycles and possibly the '}, {'순위': 8, 'ticket_id': 'b6452d1', 'Cosine 유사도': np.float64(0.7596), '문서 요약': 'The user suggests enabling remote start without needing to push the start button on the appliance fi'}, {'순위': 9, 'ticket_id': 'f11cf93', 'Cosine 유사도': np.float64(0.7582), '문서 요약': 'The user suggests allowing the normal mode to be customized to include wrinkle prevention all the ti'}, {'순위': 10, 'ticket_id': '0b178a7', 'Cosine 유사도': np.float64(0.7581), '문서 요약': 'The user suggests adding a feature to the drying cycle that allows choosing both temperature and dry'}], 
#  'self_consistency_responses': [AIMessage(content='[Response #1]\n- Selected Document: 4d1e216\n- Functional Category Alignment (Context Entity Recall): 23 points\n- Claim Coverage: 22 points\n- Evidence Faithfulness: 23 points\n- Explanation Flow Similarity: 21 points\n- Total Evaluation Score: 89 points\n- Clarity of Criteria Application: 28 points\n- Logical Coherence: 27 points\n- Persuasiveness of Explanation: 17 points\n- Absence of Ambiguity: 19 points\n- Thinking Confidence (Self-Eval): 91%', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 119, 'prompt_tokens': 669, 'total_tokens': 788, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': None, 'id': 'chatcmpl-BsOArSH1SKPF3rrMSwANvWEXCdE5C', 'finish_reason': 'stop', 'logprobs': None}, id='run--de18cca9-d921-4ab1-a08d-87a97dc8ac3f-0', usage_metadata={'input_tokens': 669, 'output_tokens': 119, 'total_tokens': 788, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}), AIMessage(content='[Response #1]\n- Selected Document: 4d1e216\n- Functional Category Alignment (Context Entity Recall): 23 points\n- Claim Coverage: 22 points\n- Evidence Faithfulness: 23 points\n- Explanation Flow Similarity: 21 points\n- Total Evaluation Score: 89 points\n- Clarity of Criteria Application: 28 points\n- Logical Coherence: 27 points\n- Persuasiveness of Explanation: 18 points\n- Absence of Ambiguity: 19 points\n- Thinking Confidence (Self-Eval): 92%', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 119, 'prompt_tokens': 669, 'total_tokens': 788, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': None, 'id': 'chatcmpl-BsOAtq7gcIf0QWPu0sqi1XgtErN49', 'finish_reason': 'stop', 'logprobs': None}, id='run--0a277b27-5957-4af2-bfd7-100309dcb5b0-0', usage_metadata={'input_tokens': 669, 'output_tokens': 119, 'total_tokens': 788, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}), AIMessage(content='[Response #1]\n- Selected Document: 4d1e216\n- Functional Category Alignment (Context Entity Recall): 23 points\n- Claim Coverage: 22 points\n- Evidence Faithfulness: 23 points\n- Explanation Flow Similarity: 21 points\n- Total Evaluation Score: 89 points\n- Clarity of Criteria Application: 28 points\n- Logical Coherence: 27 points\n- Persuasiveness of Explanation: 17 points\n- Absence of Ambiguity: 19 points\n- Thinking Confidence (Self-Eval): 91%', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 119, 'prompt_tokens': 669, 'total_tokens': 788, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': None, 'id': 'chatcmpl-BsOAu84UKAPv5Iyj2apowuShjYzOu', 'finish_reason': 'stop', 'logprobs': None}, id='run--9dbc2f99-58aa-4536-80c0-5a70234dbd5f-0', usage_metadata={'input_tokens': 669, 'output_tokens': 119, 'total_tokens': 788, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})], 
#  'parsed_self_consistency': [{'선택 문서': '', '기능 범주 일치성 (Context Entity Recall)': 0, 'Claim 커버리지': 0, '근거 충실도 (Faithfulness)': 0, '설명 흐름 유사성': 0, '평가 총점 합계': 0, '기준 적용 명확성': 0, '논리성': 0, '설명의 설득력': 0, '모호성 없음': 0, '사고 신뢰도': 0}, {'선택 문서': '', '기능 범주 일치성 (Context Entity Recall)': 0, 'Claim 커버리지': 0, '근거 충실도 (Faithfulness)': 0, '설명 흐름 유사성': 0, '평가 총점 합계': 0, '기준 적용 명확성': 0, '논리성': 0, '설명의 설득력': 0, '모호성 없음': 0, '사고 신뢰도': 0}, {'선택 문서': '', '기능 범주 일치성 (Context Entity Recall)': 0, 'Claim 커버리지': 0, '근거 충실도 (Faithfulness)': 0, '설명 흐름 유사성': 0, '평가 총점 합계': 0, '기준 적용 명확성': 0, '논리성': 0, '설명의 설득력': 0, '모호성 없음': 0, '사고 신뢰도': 0}], 
#  'self_consistency_table': '| 평가 항목 | 1번째 응답 | 2번째 응답 | 3번째 응답 |\n| --- | --- | --- | --- |\n| 선택 문서 |  |  |  |\n| 기능 범주 일치성 (Context Entity Recall) | 0 | 0 | 0 |\n| Claim 커버리지 | 0 | 0 | 0 |\n| 근거 충실도 (Faithfulness) | 0 | 0 | 0 |\n| 설명 흐름 유사성 | 0 | 0 | 0 |\n| 평가 총점 합계 | 0 | 0 | 0 |\n| 기준 적용 명확성 | 0 | 0 | 0 |\n| 논리성 | 0 | 0 | 0 |\n| 설명의 설득력 | 0 | 0 | 0 |\n| 모호성 없음 | 0 | 0 | 0 |\n| 사고 신뢰도 | 0 | 0 | 0 |', 
#  'final_result': "> ✅ Final Recommended Document: 4d1e216  \n> 📄 Selected Document Summary: The user suggests adding an option to indicate when the lint screen has been cleaned to make the reminder more effective.  \n> 🔒 Recommendation Confidence (%): 91.33%  \n> 📊 RAG-based Similarity (%): 77.96%  \n> 🧠 Selection Rationale Summary:  \n> - Same document repeatedly selected in Self-Consistency responses (+10 points)  \n> - Highest average confidence and evaluation scores: The document 4d1e216 was consistently selected in all three Self-Consistency responses, with an average confidence of 91.33%.  \n> - Superior RAG similarity and Faithfulness/Context Recall scores: It has a high RAG-based cosine similarity score of 77.96%, indicating strong alignment with the suggestion statement.  \n> - Most similar functional purpose and narrative style to the Suggestion statement: The document's content aligns well with the functional purpose of improving user interaction with the appliance, similar to the suggestion statement."
# }

## COT

In [ ]:
import pandas as pd
import os
from openai import OpenAI
from new_keyword_pipeline import setup_vectordb, search_top10_keywords, generate_new_keyword
from cot import main as sim_prompt_main

os.environ["OPENAI_API_KEY"] = "sk-"
client = OpenAI()
df = pd.read_csv('./all_origin_updated.csv', encoding='utf-8-sig')

keyword_vectordb = setup_vectordb("./keyword_list.csv")

input = {
    "components": "로봇청소기",
    "generated_summary": "ThinkQ 평면도상에 선을 그어 청소구역을 지정하도록 해주세요"
}

# sim_prompt_main에서 ICC/Proposal/유사도/키워드 추출
result, icc_df = sim_prompt_main(df, input)

ticket_id = result['ticket_id']
components = result['components']
beforechange = result['before_change']
afterchange = result['after_change']
gen_sum = result['generated_summary']
ragas_score = result['ragas_score']
sim_keyword = result['keyword']
    
print(f"\n\n✅ [최종 결과]\n"
      f"ticket_id: {ticket_id}\n"
      f"components: {components}\n"
      f"beforechange: {beforechange}\n"
      f"afterchange: {afterchange}\n"
      f"gen_sum: {gen_sum}\n"
      f"RAGAS_score: {ragas_score}\n"
      f"sim_keyword: {sim_keyword}")

# Proposal 분기 및 처리
if ragas_score >= 90:
    print(f"[{ticket_id}] 유사도 90↑ 기존 키워드 업로드: {sim_keyword}")
else:
    top10_keywords = search_top10_keywords(keyword_vectordb, gen_sum, top_k=10)
    result = generate_new_keyword(
        client, ticket_id, components, beforechange, afterchange,
        gen_sum, ragas_score, sim_keyword, top10_keywords
    )
    print(f"[{ticket_id}] 유사도 90↓ 새 키워드 생성: {result['new_keyword']}")

✅ 벡터 DB 구축 완료! 저장된 문서 수: 936

📥 입력 제안문:
ThinkQ 평면도상에 선을 그어 청소구역을 지정하도록 해주세요

⏱️ [유사도 검색] 시작
✅ [유사도 검색] 완료 - 소요 시간: 0.50초

⏱️ [Chain of Thought 분석] 시작
✅ [Chain of Thought 분석] 완료 - 소요 시간: 10.82초

📊 Chain of Thought 분석 결과
**Step 1: 피드백 유형 판단**

피드백 내용을 분석해보니, 사용자는 ThinkQ 로봇청소기의 기능 개선을 제안하고 있습니다. 구체적으로, 사용자는 청소구역을 지정할 수 있는 기능을 요청하고 있습니다. 이는 새로운 기능의 추가를 요청하는 것이므로 "Proposal"로 판단할 수 있습니다.

**판단 결과:** Proposal

**Step 2: 제안사항 분리 (Proposal인 경우만)**

피드백 내용은 단일 제안으로 보입니다. "ThinkQ 평면도상에 선을 그어 청소구역을 지정하도록 해주세요"라는 문장은 하나의 명확한 제안을 포함하고 있습니다. 따라서 별도의 제안 분리는 필요하지 않습니다.

**분리된 제안들:** ["ThinkQ 평면도상에 선을 그어 청소구역을 지정하도록 해주세요"]

**Step 3: 첫 번째 제안 선택**

여러 제안 중 원문에서 첫 번째로 나타나는 제안을 선택합니다. 이 경우, 피드백 내용 자체가 하나의 제안이므로 그대로 선택합니다.

**선택된 첫 번째 제안:** ThinkQ 평면도상에 선을 그어 청소구역을 지정하도록 해주세요

**Step 4: 제안문 표준화**

선택된 제안을 표준화된 형태로 요약합니다. "The user suggests"로 시작하는 영어 문장으로 변환하고, 컴포넌트 정보를 활용하여 맥락을 보강합니다.

**표준화된 제안문:** The user suggests adding a feature to the ThinkQ robot vacuum that allows users to draw lines on the floor pl